# Capstone Mushroom Classification 

The 

First we use beautifulSoup to scrape the website "svampguiden.com" to get the names of the most common mushrooms in Sweden. We will save these in a dictionary together with a label describing if they are edible or not.  

In [4]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup


In [5]:
# Specify url of mushroom website
mushroom_url = 'http://svampguiden.com/databasen/lista/'

page = urlopen(mushroom_url)
soup = BeautifulSoup(page, 'html.parser')


In [6]:
latin_html = soup.find_all(class_="vnamn")

Some of the latin names have a last part to their name "lat.sv" that makes the google image searches a lot worse. Removing this makes the names yeild a lot better results on google. Also, the names are all 2 words long so we only have to look at the first two names

In [7]:
mushroom_dict = {}
for name in latin_html:
    mushroom_list = name.get_text().split()
    mushroom_dict[" ".join(mushroom_list[0:2])] = 1

In [8]:
print(len(mushroom_dict))
print(len(latin_html))

371
377


### Scraping Google Images for pictures 

In [9]:
import requests
import re
import os
#import cookielib
import json

def get_soup(url,header):
    return BeautifulSoup(urlopen(Request(url,headers=header)),'html.parser')

def get_pictures(query):
    query = query # you can change the query for the image  here
    image_type="ActiOn"
    query= query.split()
    query='+'.join(query)
    url="https://www.google.co.in/search?q="+query+"&source=lnms&tbm=isch"
    print(url)
    #add the directory for your image here
    DIR="pictures"
    header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
    }
    soup = get_soup(url,header)


    ActualImages=[]# contains the link for Large original images, type of  image
    for a in soup.find_all("div",{"class":"rg_meta"}):
        link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
        ActualImages.append((link,Type))

    print("there are total {} images".format(len(ActualImages)))

    if not os.path.exists(DIR):
        os.mkdir(DIR)
    DIR = os.path.join(DIR, query.split()[0])

    if not os.path.exists(DIR):
        os.mkdir(DIR)
        ###print images
        for i , (img , Type) in enumerate( ActualImages):

            try:
                #req = request(img, headers={'User-Agent' : header})
                raw_img = urlopen(img).read()

                cntr = len([i for i in os.listdir(DIR) if image_type in i]) + 1
                #print(cntr)
                if len(Type)==0:
                    f = open(os.path.join(DIR , image_type + "_"+ str(cntr)+".jpg"), 'wb')
                else :
                    f = open(os.path.join(DIR , image_type + "_"+ str(cntr)+"."+Type), 'wb')


                f.write(raw_img)
                f.close()
            except Exception as e:
                print("could not load : {}".format(img))
                print(e)
    else: 
        print(DIR)

In [ ]:
for idx, name in enumerate(mushroom_dict): 
    print("Downloading pictures for {}".format(name))
    get_pictures(name)
    print("Done with iteration {}: {} ".format(idx, name))

https://www.google.co.in/search?q=Leotia+lubrica&source=lnms&tbm=isch
there are total 100 images
could not load : http://wpamushroomclub.org/wp/wp-content/uploads/2017/06/Leotia-lubrica.-Close-up.-By-Richard-Jacob.jpg
HTTP Error 404: Not Found
could not load : http://i1.treknature.com/photos/482/leotia.jpg
HTTP Error 403: Forbidden
Done with iteration 0: Leotia lubrica 
https://www.google.co.in/search?q=Cyathus+striatus&source=lnms&tbm=isch
there are total 100 images
could not load : http://timwheelerphotography.com/wp-content/uploads/2014/10/Cyathus_striatus_F2008.jpg
HTTP Error 403: Forbidden
Done with iteration 1: Cyathus striatus 
https://www.google.co.in/search?q=Dumontinia+tuberosa&source=lnms&tbm=isch
there are total 100 images
Done with iteration 2: Dumontinia tuberosa 
https://www.google.co.in/search?q=Aureoboletus+gentilis&source=lnms&tbm=isch
there are total 100 images
could not load : http://www.igoterra.com/photo/78/020358.JPG
HTTP Error 403: Forbidden
could not load : htt

## Preprocessing 
The pictures are now downloaded and we should preprocess them to be able to feed them into the keras model 